In [1]:
import numpy as np
import torch
from comet_ml import Experiment,ExistingExperiment
from comet_ml.integration.pytorch import log_model
import yaml
import h5py
import matplotlib.pyplot as plt

/Users/matthiasvigl/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from Finetune_hep.python import train,helpers,models

In [3]:
device = helpers.get_device()
data_config_Xbb = 'config/ParT_Xbb_config.yaml'
data_config_latent = 'config/ParT_latent_config.yaml'
data_config_latent_hlf = 'config/ParT_latent_hlf_config.yaml'
data_config_Xbb_hlf = 'config/ParT_Xbb_hlf_config.yaml'

In [4]:
model_Xbb = models.full_model(data_config_Xbb,for_inference=False)  
model_latent = models.full_model(data_config_latent,for_inference=False)  
model_Xbb_hlf = models.full_model(data_config_Xbb_hlf,concatenate_hlf=True,for_inference=True,save_representaions=True)  
model_latent_hlf = models.full_model(data_config_latent_hlf,concatenate_hlf=True,for_inference=True,save_representaions=True)  

In [5]:
model_Xbb.Task

'Xbb'

In [6]:
model_latent_hlf = models.head(data_config_latent_hlf,concatenate_hlf=True,for_inference=False)  

In [7]:
with h5py.File('../data/Data_val_sig_30.h5', 'r') as sample_bkg:
    data = {}
    print(sample_bkg.keys())
    print(sample_bkg['X_jet'])
    print(sample_bkg['X_jet_singlejet'])
    print(sample_bkg['X_pfo_singlejet'])
    data['X_jet'] = sample_bkg['X_jet'][:]
    data['X_pfo'] = sample_bkg['X_pfo'][:]
    data['labels'] = sample_bkg['labels'][:]
    data['jet_mask'] = sample_bkg['jet_mask'][:]
    inputs = helpers.build_features_and_labels(data)
    data_Xbb = {}
    data_Xbb['X_jet'] = sample_bkg['X_jet_singlejet'][:]
    data_Xbb['X_pfo'] = sample_bkg['X_pfo_singlejet'][:]
    data_Xbb['labels'] = sample_bkg['X_label_singlejet'][:]
    inputs_Xbb = helpers.build_features_and_labels_Xbb(data_Xbb)


<KeysViewHDF5 ['X_jet', 'X_jet_singlejet', 'X_label', 'X_label_singlejet', 'X_pfo', 'X_pfo_singlejet', 'jet_mask', 'labels']>
<HDF5 dataset "X_jet": shape (26, 5, 6), type "<f8">
<HDF5 dataset "X_jet_singlejet": shape (60, 6), type "<f8">
<HDF5 dataset "X_pfo_singlejet": shape (60, 100, 15), type "<f8">


/Users/matthiasvigl/Library/Python/3.9/lib/python/site-packages/Finetune_hep/python/definitions.py:74: RuntimeWarning: divide by zero encountered in log
  result = np.log(ma_data)


In [8]:
inputs.keys()

dict_keys(['pf_points', 'pf_features', 'pf_vectors', 'pf_mask', 'jet_mask', 'label', 'hl_feats'])

In [9]:
pf_points = torch.tensor(inputs['pf_points']).float().to(device)
pf_features = torch.tensor(inputs['pf_features']).float().to(device)
pf_vectors = torch.tensor(inputs['pf_vectors']).float().to(device)
pf_mask = torch.tensor(inputs['pf_mask']).float().to(device)
hl_feats = torch.tensor(inputs['hl_feats']).float().to(device)
jet_mask = torch.tensor(inputs['jet_mask']).float().to(device)

preds = model_latent(pf_points,pf_features,pf_vectors,pf_mask,jet_mask,hl_feats)

/Users/matthiasvigl/Library/Python/3.9/lib/python/site-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [40]:
preds.shape

torch.Size([26, 1])

In [11]:
pf_points = torch.tensor(inputs_Xbb['pf_points']).float().to(device)
pf_features = torch.tensor(inputs_Xbb['pf_features']).float().to(device)
pf_vectors = torch.tensor(inputs_Xbb['pf_vectors']).float().to(device)
pf_mask = torch.tensor(inputs_Xbb['pf_mask']).float().to(device)

preds_Xbb = model_Xbb(pf_points,pf_features,pf_vectors,pf_mask)

In [12]:
preds_Xbb.shape

torch.Size([60, 128])

In [10]:
idxmap = helpers.get_idxmap('config/train.txt')
idxmap_val = helpers.get_idxmap('config/val.txt')
integer_file_map = helpers.create_integer_file_map(idxmap)
integer_file_map_val = helpers.create_integer_file_map(idxmap_val)

In [11]:
experiment = Experiment(
    api_key = 'r1SBLyPzovxoWBPDLx3TAE02O',
    project_name = 'public_test',
    workspace='mvigl',
    log_graph=True, # Can be True or False.
    auto_metric_logging=True # Can be True or False
    )
Experiment.set_name(experiment,'test')
print(experiment.get_key())
experiment.log_parameter("exp_key", experiment.get_key())

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/mvigl/public-test/c957b141e4c94b0da93449d5e9a440ee



c957b141e4c94b0da93449d5e9a440ee


In [12]:
hyper_params = {
   "learning_rate": 0.01,
   "epochs": 10,
   "batch_size": 8,
   "start_epoch": 0, 
   "num_workers": 2,
}


config = dict(    
            LR = hyper_params['learning_rate'],
            batch_size = hyper_params['batch_size'],
            epochs = hyper_params['epochs'],
            device = device,
            idxmap = idxmap,
            idxmap_val = idxmap_val,
            integer_file_map = integer_file_map,
            integer_file_map_val = integer_file_map_val,
            out_model_path = 'models/test.pt',
            start_epoch = hyper_params['start_epoch'],
            num_workers = hyper_params['num_workers'],
            experiment = experiment
        )
experiment.log_parameters(hyper_params)

In [13]:
train.train_loop(model_latent, config)

Evt task
N data :  26
N data :  308
Epoch: 0 LR: 0.0


/Users/matthiasvigl/Library/Python/3.9/lib/python/site-packages/Finetune_hep/python/definitions.py:74: RuntimeWarning: divide by zero encountered in log
  result = np.log(ma_data)
/Users/matthiasvigl/Library/Python/3.9/lib/python/site-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/Users/matthiasvigl/Library/Python/3.9/lib/python/site-packages/Finetune_hep/python/definitions.py:74: RuntimeWarning: invalid value encountered in log
  result = np.log(ma_data)


train_loss: 3.2593393325805664 | validation_loss: 3.2320830821990967
Epoch: 1 LR: 0.008


COMET WARNING: Failed to log system metrics: [sys.ram,sys.cpu,sys.load]


KeyboardInterrupt: 

In [14]:
with torch.no_grad():
    model_latent.eval()
    preds_latent = model_latent(pf_points,pf_features,pf_vectors,pf_mask,jet_mask,hl_feats)

In [15]:
z = 1/(1 + np.exp(-preds_latent)) 

In [16]:
z

tensor([[0.8167],
        [0.8982],
        [0.8992],
        [0.8975],
        [0.8175],
        [0.8992],
        [0.8169],
        [0.8169],
        [0.8173],
        [0.8163],
        [0.8173],
        [0.8996],
        [0.8176],
        [0.8168],
        [0.8994],
        [0.8171],
        [0.8171],
        [0.8990],
        [0.8142],
        [0.8173],
        [0.8161],
        [0.8169],
        [0.8166],
        [0.8169],
        [0.8161],
        [0.8991]])